In [1]:
import os
import ibis
import dlt
import pandas as pd
from dotenv import load_dotenv
from lakasirclient import LakasirClient

import plotly.express as px
import plotly.graph_objects as go

load_dotenv()
ibis.options.interactive = True

In [3]:
lakasirClient = LakasirClient(
    base_url=os.getenv("LAKASIR_BASE_URL"),
    email=os.getenv("LAKASIR_EMAIL"),
    password=os.getenv("LAKASIR_PASSWORD")
)

In [26]:
pipeline = dlt.pipeline(
    pipeline_name="lakasir",
    destination="duckdb",
    # refresh="replace",
)

In [11]:
@dlt.resource
def about():
    yield lakasirClient.about()["data"]

pipeline_info = pipeline.run(about)

In [50]:
# reference: https://github.com/DataTalksClub/data-engineering-zoomcamp/blob/main/cohorts/2024/workshops/dlt_resources/data_ingestion_workshop.md

@dlt.resource(
    write_disposition="merge",
    merge_key="id",
    columns={
        'id': {'data_type': 'bigint'},
        'member': {'data_type': 'text'},
        'member__email': {'data_type': 'text'},
        'member__joined_date': {'data_type': 'date'},
        'member__address': {'data_type': 'text'},
        'note': {'data_type': 'text'},
    }
)
def sells():
    page = 1 # initial page or last page in the notes.txt
    per_page = 30
    has_more_pages = True

    while has_more_pages:
        api = lakasirClient.transaction_sells_with_page(page=page, per_page=per_page)["data"]

        if api["meta"]["to"] is None:
            has_more_pages = False
            with open("notes.txt", 'w') as file:
                file.write(f"last page = {page}")
        else:
            for record in api["data"]:
                yield dict(
                    id=record["id"],
                    member=record["member"],
                    note=record["note"],
                    code=record["code"],
                    cashier=record["cashier"]["email"],
                    payed_money=record["payed_money"],
                    money_changes=record["money_changes"],
                    total_qty=record["total_qty"],
                    total_price=record["total_price"],
                    total_discount_per_item=record["total_discount_per_item"],
                    discount=record["discount"],
                    total_discount=record["total_discount"],
                    grand_total_price=record["grand_total_price"],
                    total_cost=record["total_cost"],
                    created_at=record["created_at"],
                    updated_at=record["updated_at"],
                )
            page += 1

pipeline_info = pipeline.run(sells)

In [51]:
@dlt.resource(write_disposition="merge", merge_key="id")
def sell_details():
    page = 1 # initial page or last page in the notes.txt
    per_page = 30
    has_more_pages = True

    while has_more_pages:
        api = lakasirClient.transaction_sells_with_page(page=page, per_page=per_page)["data"]

        if api["meta"]["to"] is None:
            has_more_pages = False
        else:
            for list in api["data"]:
                for record in list["selling_details"]:
                    yield dict(
                        id=record["id"],
                        selling_id=record["selling_id"],
                        product_id=record["product_id"],
                        unit=record["product"]["unit"],
                        type=record["product"]["type"],
                        name=record["product"]["name"],
                        category=record["product"]["category"]["name"],
                        initial_price=record["product"]["initial_price"],
                        selling_price=record["product"]["selling_price"],
                        qty=record["qty"],
                        price=record["price"],
                        discount=record["discount"],
                        discount_price=record["discount_price"],
                        created_at=record["created_at"],
                        updated_at=record["updated_at"],
                    )
            page += 1

pipeline_info = pipeline.run(sell_details)

In [52]:
dataset = pipeline.dataset()

about = dataset.table("about")
sells = dataset.table("sells")
sell_details = dataset.table("sell_details")

In [23]:
START_DATE = "2025-09-01"
END_DATE = "2025-09-11"

In [53]:
sells_df = ibis.memtable(sells.df())

sells_df = sells_df.mutate(
    created_at =sells_df.created_at.cast("date"),
    updated_at =sells_df.updated_at.cast("date"),
    payed_money =sells_df.payed_money.cast("decimal(19, 2)"),
    money_changes =sells_df.money_changes.cast("decimal(19, 2)"),
    total_qty =sells_df.total_qty.cast("decimal"),
    total_price =sells_df.total_price.cast("decimal(19, 2)"),
    total_discount_per_item =sells_df.total_discount_per_item.cast("decimal(19, 2)"),
    discount =sells_df.discount.cast("decimal(19, 2)"),
    total_discount =sells_df.total_discount.cast("decimal(19, 2)"),
    grand_total_price =sells_df.grand_total_price.cast("decimal(19, 2)"),
    total_cost =sells_df.total_cost.cast("decimal(19, 2)"),
    member_name =sells_df.member__name.fill_null("Guest"),
)

sells_df = sells_df.select("id", "member_name", "code", "cashier", "payed_money", "money_changes", "total_qty", "total_price", "total_discount_per_item", "discount", "total_discount", "grand_total_price", "total_cost", "created_at", "updated_at")

In [28]:
sells_df = sells_df.filter(
    sells_df.created_at.between(START_DATE, END_DATE)
)

In [55]:
def sells_agg(group_by: list[str]) -> ibis.Table:
    return (
        sells_df.group_by(group_by)
        .aggregate(
            payed_money=sells_df.payed_money.sum(),
            money_changes=sells_df.money_changes.sum(),
            total_qty=sells_df.total_qty.count(),
            total_price=sells_df.total_price.sum(),
            total_discount_per_item=sells_df.total_discount_per_item.sum(),
            discount=sells_df.discount.sum(),
            total_discount=sells_df.total_discount.sum(),
            grand_total_price=sells_df.grand_total_price.sum(),
            total_cost=sells_df.total_cost.sum(),
        )
    )

In [56]:
# total revenue
total_revenue = sells_df.grand_total_price.sum()
# total cogs
total_cost = sells_df.total_cost.sum()
# total discounts
total_discount = sells_df.total_discount.sum()
# gross profit
gross_profit = total_revenue - (total_cost + total_discount)

In [57]:
sell_details_df = ibis.memtable(sell_details.df())

sell_details_df = sell_details_df.mutate(
    created_at =sell_details_df.created_at.cast("date"),
    updated_at =sell_details_df.updated_at.cast("date"),
    qty =sell_details_df.qty.cast("decimal"),
    initial_price =sell_details_df.initial_price.cast("decimal(19, 2)"),
    selling_price =sell_details_df.selling_price.cast("decimal(19, 2)"),
    price =sell_details_df.price.cast("decimal(19, 2)"),
    discount =sell_details_df.discount.cast("decimal(19, 2)"),
    discount_price =sell_details_df.discount_price.cast("decimal(19, 2)"),
)

sell_details_df = sell_details_df.select("id", "selling_id", "product_id", "unit", "type", "name", "category", "initial_price", "selling_price", "qty", "price", "discount", "discount_price", "created_at", "updated_at")

In [32]:
sell_details_df = sell_details_df.filter(
    sell_details_df.created_at.between(START_DATE, END_DATE)
)

In [58]:
def sell_details_agg(group_by: list[str]) -> ibis.Table:
    return (
        sell_details_df.group_by(group_by)
        .aggregate(
            qty=sell_details_df.qty.sum(),
            price=sell_details_df.price.sum(),
            discount=sell_details_df.discount.sum(),
            discount_price=sell_details_df.discount_price.sum(),
        )
    )

In [59]:
# revenue by member
sells_agg_by_member = sells_agg(["member_name"])

# revenue by product category
sell_details_agg_by_category = sell_details_agg(["category"])

# revenue and discount by product
sell_details_agg_by_name = sell_details_agg(["name"])

In [60]:
# revenue by member
revenue_by_member = sells_agg_by_member.mutate(
    total_pendapatan=sells_agg_by_member.grand_total_price,
    rasio_pendapatan=(sells_agg_by_member.grand_total_price / sells_agg_by_member.grand_total_price.sum().as_scalar()) * 100
)

revenue_by_member_pd = revenue_by_member.to_pandas()

revenue_by_member_pd_plot = px.pie(
    revenue_by_member_pd, values='rasio_pendapatan', names='member_name',
    color_discrete_sequence=px.colors.sequential.Magenta,
)

revenue_by_member_pd_plot.update_layout(
    title=f"Rasio Pendapatan Berdasarkan Keanggotaan",
    title_font_size=15,
    width=400,
    height=250,
    margin=dict(l=20, r=20, t=35, b=20),

)
revenue_by_member_pd_plot.show()

In [61]:
# revenue by category
revenue_by_category = sell_details_agg_by_category.mutate(
    total_pendapatan=sell_details_agg_by_category.discount_price,
    rasio_pendapatan=(sell_details_agg_by_category.discount_price / sell_details_agg_by_category.discount_price.sum().as_scalar()) * 100
)

revenue_by_category_pd = revenue_by_category.to_pandas()

revenue_ratio_by_category_plot = px.pie(
    revenue_by_category_pd, values='rasio_pendapatan', names='category',
    color_discrete_sequence=px.colors.sequential.Aggrnyl,
)

revenue_ratio_by_category_plot.update_layout(
    title=f"Rasio Pendapatan Berdasarkan Kategori",
    title_font_size=15,
    width=400,
    height=250,
    margin=dict(l=20, r=20, t=35, b=20),

)
revenue_ratio_by_category_plot.show()

In [62]:
# revenue by item name
revenue_by_item_name = sell_details_agg_by_name.mutate(
    total_pendapatan=sell_details_agg_by_name.discount_price,
    rasio_pendapatan=(sell_details_agg_by_name.discount_price / sell_details_agg_by_name.discount_price.sum().as_scalar()) * 100
)

revenue_by_item_name_pd = revenue_by_item_name.to_pandas()

revenue_ratio_by_item_name_plot = px.pie(
    revenue_by_item_name_pd, values='rasio_pendapatan', names='name',
    color_discrete_sequence=px.colors.sequential.Oryel,
)

revenue_ratio_by_item_name_plot.update_layout(
    title=f"Rasio Pendapatan Berdasarkan Produk",
    title_font_size=15,
    width=400,
    height=250,
    margin=dict(l=20, r=20, t=35, b=20),

)
revenue_ratio_by_item_name_plot.show()

In [64]:
# discount by item name
discount_by_item_name = sell_details_agg_by_name.mutate(
    total_potongan=sell_details_agg_by_name.discount,
    rasio_potongan=(sell_details_agg_by_name.discount / sell_details_agg_by_name.discount.sum().as_scalar()) * 100
)

discount_by_item_name_pd = discount_by_item_name.to_pandas()

discount_ratio_by_item_name_plot = px.pie(
    discount_by_item_name_pd, values='rasio_potongan', names='name',
    color_discrete_sequence=px.colors.sequential.Tealgrn,
)

discount_ratio_by_item_name_plot.update_layout(
    title=f"Rasio Potongan Harga Berdasarkan Produk",
    title_font_size=15,
    width=400,
    height=250,
    margin=dict(l=20, r=20, t=35, b=20),

)
discount_ratio_by_item_name_plot.show()

In [65]:
# 10 top best-selling product
top_sells = (
    sell_details_df.group_by(["name"]).aggregate(
        qty=sell_details_df.qty.sum(),
    ).order_by(ibis.desc("qty"))
).head(10)

top_sells_pd = top_sells.to_pandas()

top_sells_plot = px.funnel_area(
    names=top_sells_pd["name"],
    values=top_sells_pd["qty"],
    color_discrete_sequence=px.colors.sequential.Teal,
)

top_sells_plot.update_layout(
    title=f"Top 10 Produk Paling Laris",
    title_font_size=15,
    width=400,
    height=250,
    margin=dict(l=20, r=20, t=35, b=20),

)
top_sells_plot.show()

In [42]:
ITEM_NAME = ""
STEP_PREDICTION = 3

In [43]:
# sell price ratio
sell_price_ratio_by_name = (
    sell_details_df.filter(
        sell_details_df.name == ITEM_NAME
    ).group_by(["name", "selling_price"]).aggregate(
        total_discount=sell_details_df.discount.sum(),
        total_price=sell_details_df.price.sum(),
    )
)

sell_price_ratio_by_name = (
    sell_price_ratio_by_name.mutate(
        total_price_ratio=(sell_price_ratio_by_name.total_price / sell_price_ratio_by_name.total_price.sum().as_scalar()) * 100)
)

In [ ]:
sell_price_ratio_by_name_pd = sell_price_ratio_by_name.to_pandas()

sell_price_ratio_by_name_plot = px.pie(
    sell_price_ratio_by_name_pd, values='total_price_ratio', names='selling_price',
    color_discrete_sequence=px.colors.sequential.Purp,
)

sell_price_ratio_by_name_plot.update_layout(
    title=f"Rasio Harga Jual Berdasarkan Produk {ITEM_NAME}",
    title_font_size=15,
    width=400,
    height=250,
    margin=dict(l=20, r=20, t=35, b=20),

)
sell_price_ratio_by_name_plot.show()

In [44]:
sell_forecast = (
    sell_details_agg(["created_at", "name"])
    .select("created_at", "name", "qty")
)

sell_forecast_pd = (
    sell_forecast.filter(
        sell_forecast.name == ITEM_NAME
    )
    .to_pandas().astype({
        "created_at": "datetime64[ns]",
        "name": "category",
        "qty": "int32"
    })
)

date_index = pd.DatetimeIndex(sell_forecast_pd.created_at)
df_with_index = sell_forecast_pd.set_index(date_index)

start_date = date_index.min()
end_date = date_index.max()
# set specify datetime with frequency
complete_date_range = pd.date_range(start=start_date, end=end_date, freq='D')

# reindexing
sell_forecast_pd = df_with_index.reindex(complete_date_range)

# fill the rows
sell_forecast_pd = sell_forecast_pd.fillna({
    'name': sell_forecast_pd["name"].bfill(),
    'qty': 0.0
})

ValueError: Neither `start` nor `end` can be NaT

In [45]:
# Modelling and Forecasting
from lightgbm import LGBMRegressor
from skforecast.recursive import ForecasterRecursive

# create forecaster
forecaster = ForecasterRecursive(
    regressor=LGBMRegressor(random_state=int(sell_forecast_pd.name.size / 2), verbose=-1),
    lags=(sell_forecast_pd.name.size - 1)
)

forecaster.fit(y=sell_forecast_pd['qty'])

predictions = forecaster.predict(steps=STEP_PREDICTION, last_window=None)
predictions = pd.DataFrame(data=predictions)

D:\Repositories\python-notebooks\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning:

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html



ValueError: Minimum value of lags allowed is 1.

In [ ]:
sell_forecast_plot = go.Figure()

sell_forecast_plot_trace1 = go.Scatter(x=sell_forecast_pd.index, y=sell_forecast_pd["qty"], name="train", mode="lines")
sell_forecast_plot_trace2 = go.Scatter(x=predictions.index, y=predictions["pred"], name="prediction", mode="lines")
sell_forecast_plot.add_trace(sell_forecast_plot_trace1)
sell_forecast_plot.add_trace(sell_forecast_plot_trace2)

sell_forecast_plot.update_layout(
    title=f"Prediksi Penjualan Produk {ITEM_NAME}",
    title_font_size=16,
    xaxis_title="Tanggal",
    yaxis_title="Total Penjualan",
    height=400,
    margin=dict(l=20, r=20, t=35, b=20),
    legend=dict(orientation="h", yanchor="top", y=1.01, xanchor="left", x=0)
)
sell_forecast_plot.show()

In [ ]:
mean_of_sell = sum(predictions["pred"].to_numpy()) / len(predictions["pred"].to_numpy())

print(f"Hasil prediksi menunjukan dibutuhkan rata-rata {mean_of_sell} buah item yang siap jual untuk produk {ITEM_NAME}, sehingga memenuhi target penjualan dalam {STEP_PREDICTION} hari kedepan.")